<a href="https://colab.research.google.com/github/fjadidi2001/Insurance/blob/main/ClaimYN_Prediction_TabNet_80result.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Specify file path
file_path = '/content/drive/My Drive/Insurance/telematics_syn.csv'

# Import pandas (assuming you want to use it to read the CSV)
import pandas as pd

# Read the CSV file
df = pd.read_csv(file_path)

print(df.shape)  # Should print (100000, 52)

print(df.head())  # Example: print the first few rows of the dataframe


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
(100000, 52)
   Duration  Insured.age Insured.sex  Car.age  Marital  Car.use  Credit.score  \
0       366           45        Male       -1  Married  Commute         609.0   
1       182           44      Female        3  Married  Commute         575.0   
2       184           48      Female        6  Married  Commute         847.0   
3       183           71        Male        6  Married  Private         842.0   
4       183           84        Male       10  Married  Private         856.0   

  Region  Annual.miles.drive  Years.noclaims  ...  Left.turn.intensity10  \
0  Urban             6213.71              25  ...                    1.0   
1  Urban            12427.42              20  ...                   58.0   
2  Urban            12427.42              14  ...                    0.0   
3  Urban             6213.71              43  ...                  

#step 2

## pre1

In [5]:
# Encoding categorical columns using one-hot encoding
categorical_cols = ['Marital', 'Insured.sex', 'Car.use', 'Region', 'Territory']
df = pd.get_dummies(df, columns=categorical_cols)

# Creating the ClaimYN variable
df['ClaimYN'] = df['NB_Claim'].apply(lambda x: 1 if x > 0 else 0)

# Saving the preprocessed data to a new file
preprocessed_file_path = '/content/drive/My Drive/pre_telematics_syn.csv'
df.to_csv(preprocessed_file_path, index=False)
print(df.shape)  # Should print (100000, number of columns after encoding)


(100000, 113)


## pre2

In [6]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

# Feature columns (excluding response columns)
feature_cols = [col for col in df.columns if col not in ['NB_Claim', 'AMT_Claim', 'ClaimYN']]

# Applying StandardScaler and MinMaxScaler
scaler = StandardScaler()
minmax_scaler = MinMaxScaler()

df[feature_cols] = scaler.fit_transform(df[feature_cols])
df[feature_cols] = minmax_scaler.fit_transform(df[feature_cols])

# Save the preprocessed data to a new file
preprocessed_file_path_scaled = '/content/drive/My Drive/pre_telematics_syn_scaled.csv'
df.to_csv(preprocessed_file_path_scaled, index=False)
print(df.shape)  # Should print (100000, number of columns after scaling and normalization)


(100000, 113)


## pre3

In [7]:
# Advanced feature engineering steps, if any
# For example, aggregating harsh driving events:
# Check if the columns exist before using them
if all(col in df.columns for col in ['Accel.xx.miles', 'Brake.xx.miles', 'Left.turn.intensityxx', 'Right.turn.intensityxx']):
    df['HarshDriving'] = df['Accel.xx.miles'] + df['Brake.xx.miles'] + df['Left.turn.intensityxx'] + df['Right.turn.intensityxx']
else:
    print("Warning: One or more columns required for 'HarshDriving' calculation are missing.")

# Save the fully preprocessed data to a new file
preprocessed_file_path_final = '/content/drive/My Drive/pre_telematics_syn_advanced.csv'
df.to_csv(preprocessed_file_path_final, index=False)
print(df.shape)  # Should print (100000, number of columns after feature engineering)

(100000, 113)


In [8]:
from sklearn.model_selection import train_test_split

# Define the response and feature columns
response_cols = ['NB_Claim', 'AMT_Claim', 'ClaimYN']
feature_cols = [col for col in df.columns if col not in response_cols]

# Split into 80% train and 20% test
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Further split train into train and validation
train_df, val_df = train_test_split(train_df, test_size=0.2, random_state=42)

print(train_df.shape, val_df.shape, test_df.shape)


(64000, 113) (16000, 113) (20000, 113)


In [9]:
!pip install pytorch-tabnet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.5/44.5 kB 2.3 MB/s eta 0:00:00


In [10]:
# Install the library
!pip install pytorch-tabnet

# Import required libraries
from pytorch_tabnet.tab_model import TabNetClassifier
import numpy as np

# Prepare the data for TabNet
X_train = train_df[feature_cols].values
y_train = train_df['ClaimYN'].values
X_val = val_df[feature_cols].values
y_val = val_df['ClaimYN'].values
X_test = test_df[feature_cols].values
y_test = test_df['ClaimYN'].values

# Create and train the TabNet model
clf = TabNetClassifier()

clf.fit(
    X_train, y_train,
    eval_set=[(X_train, y_train), (X_val, y_val)],
    eval_name=['train', 'val'],
    eval_metric=['accuracy'],
    max_epochs=50,
    patience=10,
    batch_size=1024,
    virtual_batch_size=128,
    num_workers=0,
    weights=1,
    drop_last=False
)


/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 0.64251 | train_accuracy: 0.17378 | val_accuracy: 0.17856 |  0:00:08s
epoch 1  | loss: 0.58654 | train_accuracy: 0.25545 | val_accuracy: 0.25669 |  0:00:12s
epoch 2  | loss: 0.56904 | train_accuracy: 0.39398 | val_accuracy: 0.39744 |  0:00:17s
epoch 3  | loss: 0.56708 | train_accuracy: 0.45408 | val_accuracy: 0.4575  |  0:00:23s
epoch 4  | loss: 0.56379 | train_accuracy: 0.57734 | val_accuracy: 0.57788 |  0:00:28s
epoch 5  | loss: 0.55983 | train_accuracy: 0.51839 | val_accuracy: 0.51962 |  0:00:34s
epoch 6  | loss: 0.55754 | train_accuracy: 0.61255 | val_accuracy: 0.61325 |  0:00:39s
epoch 7  | loss: 0.54772 | train_accuracy: 0.6257  | val_accuracy: 0.62606 |  0:00:44s
epoch 8  | loss: 0.53942 | train_accuracy: 0.63492 | val_accuracy: 0.63256 |  0:00:50s
epoch 9  | loss: 0.53525 | train_accuracy: 0.63497 | val_accuracy: 0.63038 |  0:00:55s
epoch 10 | loss: 0.53837 | train_accuracy: 0.63761 | val_accuracy: 0.63319 |  0:01:00s
epoch 11 | loss: 0.52921 | train_accuracy: 

/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


In [11]:
import time
import ast
import traceback

# Function to evaluate functionality and handle errors
def evaluate_functionality(code: str, *args):
    try:
        exec_globals = {}
        exec(code, exec_globals)

        if 'main' in exec_globals:
            start_time = time.time()
            result = exec_globals['main'](*args)  # Assuming a main function takes arguments
            end_time = time.time()
            return {
                'result': result,
                'execution_time': end_time - start_time,
                'error': None
            }
        else:
            return {'result': None, 'execution_time': None, 'error': 'No main function found'}
    except Exception as e:
        return {'result': None, 'execution_time': None, 'error': str(e)}

# Function to evaluate readability (basic check)
def evaluate_readability(code: str):
    try:
        # Check for indentation errors using AST
        ast.parse(code)
        lines = code.split('\n')
        length = sum(len(line) for line in lines)
        avg_length = length / len(lines) if lines else 0
        return {
            'line_count': len(lines),
            'average_length_per_line': avg_length,
            'error': None
        }
    except SyntaxError as e:
        return {'line_count': None, 'average_length_per_line': None, 'error': str(e)}

# Main evaluation function
def evaluate_code(code: str, *args):
    print("Evaluating functionality...")
    functionality_results = evaluate_functionality(code, *args)

    print("Evaluating readability...")
    readability_results = evaluate_readability(code)

    return {
        'functionality': functionality_results,
        'readability': readability_results
    }

# Example usage:
if __name__ == "__main__":
    sample_code = """
def main(x):
    return x * x  # Simple function to square a number
    """

    evaluation_results = evaluate_code(sample_code, 5)
    print(evaluation_results)

Evaluating functionality...
Evaluating readability...
{'functionality': {'result': 25, 'execution_time': 1.1920928955078125e-06, 'error': None}, 'readability': {'line_count': 4, 'average_length_per_line': 19.0, 'error': None}}


In [12]:
import time
import ast
import traceback
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

# Function to evaluate functionality and handle errors
def evaluate_functionality(code: str, *args):
    try:
        exec_globals = {}
        exec(code, exec_globals)

        # Assuming the code defines a function named 'model_predict' and 'true_labels'
        predictions = exec_globals['model_predict'](*args)
        true_labels = exec_globals['true_labels']

        # Generate confusion matrix
        cm = confusion_matrix(true_labels, predictions)
        disp = ConfusionMatrixDisplay(confusion_matrix=cm)

        # Display the confusion matrix
        disp.plot(cmap=plt.cm.Blues)
        plt.title('Confusion Matrix')
        plt.show()

        return predictions  # Return predictions for further use if needed
    except Exception as e:
        print("An error occurred:")
        traceback.print_exc()

# Sample code to evaluate
sample_code = """
import numpy as np

# Simulating true labels and a model prediction function
true_labels = [0, 1, 0, 1, 0, 1, 0, 1]  # Example true labels

def model_predict(*args):
    # Simple mock prediction logic (replace with actual model prediction)
    return [1 if i % 2 == 0 else 0 for i in range(len(args))]
"""

# Evaluate the sample code
evaluation_results = evaluate_functionality(sample_code, 5)
print(evaluation_results)

An error occurred:
None


Traceback (most recent call last):
  File "<ipython-input-12-efab91576aff>", line 18, in evaluate_functionality
    cm = confusion_matrix(true_labels, predictions)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/utils/_param_validation.py", line 213, in wrapper
    return func(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py", line 342, in confusion_matrix
    y_type, y_true, y_pred = _check_targets(y_true, y_pred)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py", line 103, in _check_targets
    check_consistent_length(y_true, y_pred)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py", line 457, in check_consistent_length
    raise ValueError(
ValueError: Found input variables with inconsistent numbers of samples: [8, 1]


1. load dataset from google drive
2. create claimYN from NB claim and AMT-claim
3. 3 level prepreocess
4. simplr TabNet model

result:
<br>
**max_epochs = 50 with best_epoch = 46 and best_val_accuracy = 0.78419**
